# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

## imports for Python, Pandas

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
from IPython.display import display, HTML

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## 1. Find the 10 countries with most projects


In [2]:
# load full data as Pandas dataframe
wbp_json_df = pd.read_json('data/world_bank_projects.json')

# group by country and aggregate by project count. 
project_count = wbp_json_df.groupby('countryshortname')['countrycode'].agg('count').nlargest(10)

# rename columns and print
project_count = project_count.reset_index()
project_count.rename(columns={'countryshortname':'Country', 'countrycode':'# of Projects'}, inplace=True)
project_count


,Country,# of Projects
0,China,19
1,Indonesia,19
2,Vietnam,17
3,India,16
4,"Yemen, Republic of",13
5,Bangladesh,12
6,Morocco,12
7,Nepal,12
8,Africa,11
9,Mozambique,11


***Africa*** is returned in the top 10 list. Since Africa is a continent and not a country, it should be removed. A quick look at the project names shows that these are indeed regional projects.

In [3]:
africa = wbp_json_df.loc[wbp_json_df.countryshortname == 'Africa', ['countryshortname', 'project_name']]
africa

,countryshortname,project_name
45,Africa,West Africa Regional Disease Surveillance Cap...
46,Africa,RCIP4 - Regional Communications Infrastructure...
51,Africa,"Building Resilience through Innovation, Commun..."
58,Africa,Fighting against wildlife poaching and illegal...
65,Africa,Regional Rusumo Falls Hydroelectric Project
99,Africa,Additional Financing-West Africa Agricultural ...
167,Africa,West Africa Regional Communications Infrastruc...
184,Africa,Southern Africa Trade and Transport Facilitati...
287,Africa,Agricultural Productivity Program for Southern...
353,Africa,Nile Cooperation for Results Project


After removing **_Africa_** we are now faced with an ambiguos problem statement. There is a three-way tie for the 10th country, resulting in a total of 12 countries: 

In [4]:
# as above, dropping 'Africa'
project_count = wbp_json_df.groupby('countryshortname')['countrycode'].agg('count').drop('Africa').nlargest(12)
project_count = project_count.reset_index()
project_count.rename(columns={'countryshortname':'Country', 'countrycode':'# of Projects'}, inplace=True)

# rankings
project_count['Rank'] = project_count['# of Projects'].rank(method='min', ascending=False).astype(int)
project_count.sort_values(by=['Rank'])

,Country,# of Projects,Rank
0,China,19,1
1,Indonesia,19,1
2,Vietnam,17,3
3,India,16,4
4,"Yemen, Republic of",13,5
5,Bangladesh,12,6
6,Morocco,12,6
7,Nepal,12,6
8,Mozambique,11,9
9,Brazil,9,10


Below is the final answer, assuming alphabetical ranking as secondary criteria:

In [5]:
# group by country and aggregate by project count
country_count = wbp_json_df.groupby('countryshortname')['countrycode'].agg('count').drop('Africa').nlargest(10).reset_index()

# add ranking
rank = pd.DataFrame({'Rank': range(1, 11)})
result = pd.concat([rank, country_count], axis=1)

# rename columns and display
result.rename(columns={'countryshortname':'Country', 'countrycode':'# of Projects'}, inplace=True)

display(HTML(result.to_html(index=False)))

Rank,Country,# of Projects
1,China,19
2,Indonesia,19
3,Vietnam,17
4,India,16
5,"Yemen, Republic of",13
6,Bangladesh,12
7,Morocco,12
8,Nepal,12
9,Mozambique,11
10,Brazil,9


## 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

'mjtheme_namecode' is a nested element. We can compare both ***code*** and ***name*** as criteria:

In [6]:
# load json as string
json_str = json.load((open('data/world_bank_projects.json')))

# further populate tables created from nested element
themes = json_normalize(json_str, record_path='mjtheme_namecode')

# display top 10
print(themes.code.value_counts(ascending=False)[:10])
print(themes.name.value_counts(ascending=False)[:10]) 

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64


The difference in value counts is likely due to missing ***name*** fields. Let's move on to problem 3.

## Problem 3: In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
Populate missing values using ".fillna()".

In [7]:
# replace empty fields and fill in project names
themes['name'] = themes['name'].apply(lambda x: None if x == '' else x)
projects = themes.sort_values(by=['code', 'name']).fillna(method='ffill')['name'].value_counts(ascending=False).head(10)

# concatenate, rename columns and display:
projects = projects.to_frame().reset_index()
projects = pd.concat([rank, projects], axis=1)
projects.rename(columns={'index':'Project Name', 'name':'# of Projects'}, inplace=True)
display(HTML(projects.to_html(index=False)))

Rank,Project Name,# of Projects
1,Environment and natural resources management,250
2,Rural development,216
3,Human development,210
4,Public sector governance,199
5,Social protection and risk management,168
6,Financial and private sector development,146
7,Social dev/gender/inclusion,130
8,Trade and integration,77
9,Urban development,50
10,Economic management,38
